In [6]:
# general imports
import sys
sys.path.append('/home/tplas/repos/Vape/')
sys.path.append('/home/tplas/repos/Vape/utils')
sys.path.append('/home/tplas/repos/Vape/jupyter/')
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import utils_funcs as utils
import run_functions as rf
from subsets_analysis import Subsets
import pickle
import sklearn.decomposition
from cycler import cycler
import seaborn as sns
plt.rcParams['axes.prop_cycle'] = cycler(color=sns.color_palette('colorblind'))

%run setup_notebook.ipynb

ERROR:root:File `'setup_notebook.ipynb.py'` not found.


In [12]:
class Session:
    def __init__(self, mouse, run_number, pkl_path, remove_nan_trials=True):
        self.mouse = mouse
        self.run_number = run_number
        self.pkl_path = pkl_path
        self.name = f'Mouse {mouse}, run {run_number}'
        self.run = None
        
        self.load_data()
        self.define_s1_s2()
        self.build_trials()
        self.label_trials()
        self.remove_nan_trials_inplace()
    
    def __str__(self):  
        """Define name"""
        return self.name
    
    def __repr__(self):
        """Define representation"""
        return f'instance {self.name} of Session class'
    
    def load_data(self, verbose=True):
        if verbose:
            print(f'Now loading mouse {mouse}, run {run_number}')
        run_path = os.path.join(self.pkl_path, self.mouse, f'run{self.run_number}.pkl')
        with open(run_path, 'rb') as f:  # load data
            r = pickle.load(f)
            self.run = r
        ## Start preprocessing:
        self.flu = self.run.flu
        self.tstart_galvo = utils.threshold_detect(self.run.x_galvo_uncaging, 0)
        self.trial_start = self.run.trial_start
        assert len(self.trial_start) == len(self.tstart_galvo)
        self.galvo_ms = self.run.aligner.B_to_A(self.tstart_galvo)
        if verbose:
            print('microcontroller trial starts occur on average {} ms from galvo trial starts'
              .format(round(np.mean(self.trial_start - self.galvo_ms), 2)))
        assert self.run.frames_ms.shape == self.run.flu.shape

        ## Info about PS & iutcome
        ### a different number of cells were stimulated on each trial
        ### need to create a Subsets object to get this info (future code refinement will
        ### include this info directly in the run object
        self.subsets = Subsets(self.run)
        self.trial_subsets = self.subsets.trial_subsets
        self.n_stim_arr = np.unique(self.trial_subsets)
        self.outcome = self.run.outcome
        self.outcome_arr = np.unique(self.outcome)

    def define_s1_s2(self, im_size=1024):  # define border (which is hard-defined at middle of image)
        if self.run is None:
            self.load_data()
        self.n_cells = self.run.stat.shape[0]
        av_ypix = np.zeros(self.n_cells)
        av_xpix = np.zeros(self.n_cells)
        self.plane_number = np.zeros(self.n_cells)
        for neuron_id in range(self.n_cells):
            av_xpix[neuron_id] = np.mean(self.run.stat[neuron_id]['xpix']) % im_size  # modulo 1024 because different planes are transposed by image size
            av_ypix[neuron_id] = np.mean(self.run.stat[neuron_id]['ypix']) % im_size
            self.plane_number[neuron_id] = self.run.stat[neuron_id]['iplane']
        self.s2_bool = av_ypix > 512
        self.s1_bool = np.logical_not(self.s2_bool)

    def build_trials(self, pre_frames=21, post_frames=21, verbose=True):

        print(f'number of pre frames: {pre_frames}, number of post frames {post_frames}')

        self.pre_frames = pre_frames
        self.post_frames = post_frames
        self.art_gap_start = pre_frames - 1
        self.final_pre_gap_tp = np.arange(self.art_gap_start)[-1]
        self.art_gap_stop = pre_frames + 3
        self.filter_ps_array = np.concatenate((np.arange(self.art_gap_start), 
                                      np.arange(self.art_gap_stop, pre_frames + post_frames)))  # filter out few frames around PS

        # array of fluoresence through behavioural trials (n_cells x n_trials x n_frames)
        # with e.g. the first trials spanning (galvo_ms[0] - pre_frames) : (galvo_ms[0] + post_frames)
        self.behaviour_trials = utils.build_flu_array(self.run, self.galvo_ms, pre_frames, post_frames)
        self.behaviour_trials = self.behaviour_trials - np.nanmean(self.behaviour_trials, (1, 2))[:, np.newaxis, np.newaxis]
        print(f'Shape new array : {self.behaviour_trials.shape}')
        assert self.behaviour_trials.shape[1] == self.outcome.shape[0]


        self.pre_rew_trials = utils.build_flu_array(self.run, self.run.pre_reward, pre_frames, 
                                           pre_frames, is_prereward=True)  # equal amount b/c no PS artefact
        self.pre_rew_trials = self.pre_rew_trials[:, 1:9, :]
        assert np.sum(np.isnan(self.pre_rew_trials)) == 0
        self.pre_rew_trials = self.pre_rew_trials - np.mean(self.pre_rew_trials, (1, 2))[:, np.newaxis, np.newaxis]
        print(self.behaviour_trials.shape, self.pre_rew_trials.shape)

    def label_trials(self, shuffle_labels=False):
        self.decision = np.logical_or(self.outcome == 'hit', self.outcome == 'fp').astype('int')
        self.photostim = np.ones_like(self.trial_subsets)  # ones = 5-50
        self.photostim[self.trial_subsets == 0] = 0
        self.photostim[self.trial_subsets == 150] = 2
        self.photostim_occ = {x: np.sum(self.photostim == x) for x in list(np.unique(self.photostim))}
        print(f'photo stim occurences: {self.photostim_occ}')
        # unique_ps_numbers = np.unique(trial_subsets)
        # assert unique_ps_numbers[0] == 0
        # for i_n, n_ps in enumerate(unique_ps_numbers):
        #     photostim[trial_subsets == n_ps] = i_n
        assert self.photostim.shape == self.decision.shape
        if shuffle_labels:
            np.random.shuffle(self.photostim)
            np.random.shuffle(self.decision)

        self.n_unique_stims = len(np.unique(self.photostim))
        self.n_neurons = self.behaviour_trials.shape[0]
        self.n_times = self.behaviour_trials.shape[2]
        self.n_trials = self.behaviour_trials.shape[1]
        self.n_unique_dec = len(np.unique(self.decision))
        self.occ_table = np.zeros((self.n_unique_stims, 2))  # stim x dec
        for dec in range(self.n_unique_dec):
            for stim in range(self.n_unique_stims):
                self.occ_table[stim, dec] = np.sum(np.logical_and(self.decision == dec, self.photostim == stim))
        self.n_com_trials = np.max(self.occ_table).astype('int')
        print('Occurence table:')
        print(self.occ_table)
    
    def remove_nan_trials_inplace(self, verbose=True):
        
        self.nonnan_trials = np.unique(np.where(~np.isnan(self.behaviour_trials))[1])
        self.behaviour_trials = self.behaviour_trials[:, self.nonnan_trials, :]
        self.photostim = self.photostim[self.nonnan_trials]
        self.decision = self.decision[self.nonnan_trials]
        self.trial_subsets = self.trial_subsets[self.nonnan_trials]
        
        if verbose:
            print(f'{len(self.nonnan_trials)} / {self.behaviour_trials.shape[1]} non nan trials identified')
            print(f'Numbers of PS cells: {np.unique(self.trial_subsets)}')  # exact amount of PS neurons
            print(f'Time array: {self.filter_ps_array}')  # time points outside of laser artefact


In [13]:
## Load data
pkl_path = '/mnt/qnap_jrowland/run_pkls'


# dictionary of mice and run numbers to analyse
run_dict = {
            'J048' : [27, 30, 32]  # 29
            #'RL048': [23, 24, 25, 28, 29]
           }

# local path to behaviour pickle files
# this takes a while to load so maybe should do some further caching in the future
# pkl_path = '/home/jamesrowland/Documents/Code/Vape/run_pkls/'

sessions = {}
total_ds = 0
for mouse in run_dict:
    for run_number in run_dict[mouse]:
        sessions[total_ds] = Session(mouse, run_number, pkl_path)
        total_ds += 1

Now loading mouse J048, run 27
microcontroller trial starts occur on average -36.39 ms from galvo trial starts
number of pre frames: 21, number of post frames 21
Shape new array : (2101, 143, 42)
(2101, 143, 42) (2101, 8, 42)
photo stim occurences: {0: 51, 1: 51, 2: 41}
Occurence table:
[[26. 25.]
 [25. 26.]
 [ 5. 36.]]
Now loading mouse J048, run 30
microcontroller trial starts occur on average -37.0 ms from galvo trial starts
number of pre frames: 21, number of post frames 21
Shape new array : (2085, 235, 42)
(2085, 235, 42) (2085, 8, 42)
photo stim occurences: {0: 88, 1: 84, 2: 63}
Occurence table:
[[56. 32.]
 [31. 53.]
 [ 9. 54.]]
Now loading mouse J048, run 32
microcontroller trial starts occur on average nan ms from galvo trial starts
number of pre frames: 21, number of post frames 21
Shape new array : (1972, 274, 42)
(1972, 274, 42) (1972, 8, 42)
photo stim occurences: {0: 94, 1: 82, 2: 98}
Occurence table:
[[67. 27.]
 [42. 40.]
 [31. 67.]]
